In [1]:
import numpy as np
scanners = {}
with open('./input.txt','r') as file:
    for line in file:
        if ('---') in line:
            idx = int(line.strip()[12:].split(' ')[0])
            scanners[idx] = []
        elif line.strip() == '':
            continue
        else:
            x,y,z = [int(i) for i in line.strip().split(',')]
            scanners[idx].append((x,y,z))
for k in scanners.keys():
    scanners[k] = np.array(scanners[k])
    
def deg2rad(deg):
    return deg/360*2*np.pi

#make rotation matrix for given roll, pitch, and yaw
def rx(roll):
    roll = deg2rad(roll)
    Rx = np.array([[1, 0            , 0.          ],
                   [0, np.cos(roll), -np.sin(roll)],
                   [0, np.sin(roll),  np.cos(roll)]])
    return Rx

def ry(pitch):
    pitch = deg2rad(pitch)    
    Ry = np.array([[np.cos(pitch) , 0, np.sin(pitch)],
                   [0             , 1,             0],
                   [-np.sin(pitch), 0, np.cos(pitch)]])
    return Ry
    
def rz(yaw):
    yaw = deg2rad(yaw)    
    Rz = np.array([[np.cos(yaw), -np.sin(yaw), 0],
                   [np.sin(yaw),  np.cos(yaw), 0],
                   [0.         ,  0          , 1]])
    return Rz

def rotate():
    #do the first 4 faces (6,5,1,2)
    r = np.identity(3)    
    for _ in range(4):
        r = np.matmul(r,rx(90))
        for _ in range(4):
            r = np.matmul(r,rz(90)).astype(int)
            yield r
    #do face 3
    r = np.identity(3)
    r = np.matmul(r,ry(90))
    for _ in range(4):
        r = np.matmul(r,rz(90)).astype(int)
        yield r
    #do face 4        
    r = np.matmul(r,ry(180))    
    for _ in range(4):
        r = np.matmul(r,rz(90)).astype(int)
        yield r
        
def equal(a,b):
    counter = 0
    for x in a:
        for y in b:
            if np.all(x==y):
                counter +=1
    return counter
 
    
rot = []
for r in rotate():
    rot.append(r)
rot = np.array(rot)    

In [2]:
scan = scanners.copy()
temp = scan.copy()
for k,v in scan.items():
    temp[k] = np.matmul(rot,v.T)
    #temp[k] = np.swapaxes(temp[k],2,0)
    temp[k] = np.swapaxes(temp[k],1,2) 
    temp
scan = temp.copy()
scan[0].shape

(24, 26, 3)

In [38]:
np.reshape(np.swapaxes(scan[0],1,0),(-1,3)

(624, 3)

In [148]:
keys = list(scan.keys())
s1=scanners[0].copy()

while len(keys) > 1:
    keys = list(scan.keys())
    print(len(keys))
    winning = False
    for b1 in range(len(s1))[:-11]:
        for k2 in keys[1:]:
            for s in scan[k2]:
                for b2 in range(len(s)-11):
                    temp = s - s[b2] + s1[b1]
                    x = equal(s1,temp)
                    if x >= 12:
                        winning = True
                        break
                if winning:
                    break
            if winning:
                break
            #else:
            #    print(f'\tNot winning {b1} - {k2}')      
        if winning:
            break
    if winning:
        s1 = list(s1)
        for s in temp:
            if np.any(s == s1) == False:
                s1.append(s)
        s1 = np.array(s1)
        print(f'\tWinning {k2}')
        del scan[k2]                    

37
	Winning 27
36
	Winning 26
35
	Winning 2
34
	Winning 4
33
	Winning 11
32
	Winning 12
31
	Winning 18
30
	Winning 23
29
	Winning 24
28
	Winning 31
27
	Winning 1
26
	Winning 10
25
	Winning 32
24
	Winning 30
23
	Winning 15
22
	Winning 8
21
21
21
21
21


KeyboardInterrupt: 

In [312]:
def add_r(row):
    for i,r in enumerate(rot):
        row[f"r{i}"] = np.matmul(r,row['beacon']).astype(int)
    return row
    
def align_r(row, align_to):
    for a_idx, a in enumerate(align_to):
        for r_idx,r in enumerate(rot):
            row[f"a{a_idx}r{r_idx}"] = row[f"r{r_idx}"] + a
    return row
    
d = []
import pandas as pd
for k,v in scanners.items():
    for b in v:
        d.append({
            'scanner':k, 
            'beacon':b
        })
df = pd.DataFrame(d)


df = df.apply(lambda x: add_r(x), axis = 1)
align_to = df[(df['scanner']==0)]['beacon'].copy()

df = df.apply(lambda x: align_r(x,align_to), axis = 1)

for i,r in enumerate(rot):
    df = df.drop(labels=f'r{i}',axis=1)
df

,scanner,beacon,a0r0,a0r1,a0r2,a0r3,a0r4,a0r5,a0r6,a0r7,...,a25r14,a25r15,a25r16,a25r17,a25r18,a25r19,a25r20,a25r21,a25r22,a25r23
0,0,"[-535, -526, -765]","[-9, 239, -1300]","[0, 239, -239]","[-1061, 239, -230]","[-1070, 239, -1291]","[-9, 9, 0]","[0, -1052, 0]","[-1061, -1061, 0]","[-1070, 0, 0]",...,"[-1075, 1402, -257]","[-1084, 341, -257]","[-1314, 332, -18]","[-1314, 1393, -27]","[-1314, 1402, 1034]","[-1314, 341, 1043]","[216, 332, 1034]","[216, 1393, 1043]","[216, 1402, -18]","[216, 341, -27]"
1,0,"[627, -311, 420]","[-224, -946, -138]","[-1162, -946, -454]","[-846, -946, -1392]","[92, -946, -1076]","[-224, -1153, -1185]","[-1162, -837, -1185]","[-846, 101, -1185]","[92, -215, -1185]",...,"[-860, 240, 928]","[78, 556, 928]","[-129, 1494, 197]","[-129, 1178, 1135]","[-129, 240, 819]","[-129, 556, -119]","[-969, 1494, 819]","[-969, 1178, -119]","[-969, 240, 197]","[-969, 556, 1135]"
2,0,"[-587, -471, 242]","[-64, -768, -1352]","[52, -768, -294]","[-1006, -768, -178]","[-1122, -768, -1236]","[-64, 61, -1007]","[52, -997, -1007]","[-1006, -1113, -1007]","[-1122, -55, -1007]",...,"[-1020, 1454, 750]","[-1136, 396, 750]","[-307, 280, 37]","[-307, 1338, -79]","[-307, 1454, 979]","[-307, 396, 1095]","[-791, 280, 979]","[-791, 1338, 1095]","[-791, 1454, 37]","[-791, 396, -79]"
3,0,"[581, -596, -710]","[61, 184, -184]","[-1116, 184, -169]","[-1131, 184, -1346]","[46, 184, -1361]","[61, -1107, -55]","[-1116, -1122, -55]","[-1131, 55, -55]","[46, 70, -55]",...,"[-1145, 286, -202]","[32, 271, -202]","[-1259, 1448, -88]","[-1259, 1463, 1089]","[-1259, 286, 1104]","[-1259, 271, -73]","[161, 1448, 1104]","[161, 1463, -73]","[161, 286, -88]","[161, 271, 1089]"
4,0,"[-602, 794, 452]","[-1329, -978, -1367]","[67, -978, -1559]","[259, -978, -163]","[-1137, -978, 29]","[-1329, 76, -1217]","[67, 268, -1217]","[259, -1128, -1217]","[-1137, -1320, -1217]",...,"[245, 1469, 960]","[-1151, 1661, 960]","[-97, 265, 1302]","[-97, 73, -94]","[-97, 1469, -286]","[-97, 1661, 1110]","[-1001, 265, -286]","[-1001, 73, 1110]","[-1001, 1469, 1302]","[-1001, 1661, -94]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
945,36,"[775, -516, -697]","[-19, 171, 10]","[-1310, 171, -249]","[-1051, 171, -1540]","[240, 171, -1281]","[-19, -1301, -68]","[-1310, -1042, -68]","[-1051, 249, -68]","[240, -10, -68]",...,"[-1065, 92, -189]","[226, 351, -189]","[-1246, 1642, -8]","[-1246, 1383, 1283]","[-1246, 92, 1024]","[-1246, 351, -267]","[148, 1642, 1024]","[148, 1383, -267]","[148, 92, -8]","[148, 351, 1283]"
946,36,"[-714, 717, -779]","[-1252, 253, -1479]","[179, 253, -1482]","[182, 253, -51]","[-1249, 253, -48]","[-1252, 188, 14]","[179, 191, 14]","[182, -1240, 14]","[-1249, -1243, 14]",...,"[168, 1581, -271]","[-1263, 1584, -271]","[-1328, 153, 1225]","[-1328, 150, -206]","[-1328, 1581, -209]","[-1328, 1584, 1222]","[230, 153, -209]","[230, 150, 1222]","[230, 1581, 1225]","[230, 1584, -206]"
947,36,"[-767, 719, -717]","[-1254, 191, -1532]","[232, 191, -1484]","[184, 191, 2]","[-1302, 191, -46]","[-1254, 241, -48]","[232, 193, -48]","[184, -1293, -48]","[-1302, -1245, -48]",...,"[170, 1634, -209]","[-1316, 1586, -209]","[-1266, 100, 1227]","[-1266, 148, -259]","[-1266, 1634, -211]","[-1266, 1586, 1275]","[168, 100, -211]","[168, 148, 1275]","[168, 1634, 1227]","[168, 1586, -259]"
948,36,"[707, 502, 454]","[-1037, -980, -58]","[-1242, -980, -1267]","[-33, -980, -1472]","[172, -980, -263]","[-1037, -1233, -1219]","[-1242, -24, -1219]","[-33, 181, -1219]","[172, -1028, -1219]",...,"[-47, 160, 962]","[158, 1369, 962]","[-95, 1574, 1010]","[-95, 365, 1215]","[-95, 160, 6]","[-95, 1369, -199]","[-1003, 1574, 6]","[-1003, 365, -199]","[-1003, 160, 1010]","[-1003, 1369, 1215]"


In [392]:
14+160+1400+160000

161574

In [388]:
merger = []
for i in df['scanner'].unique():#range(2,37):
    if i == 0:
        continue
    print(i)
    for j in range(26):
        for r in range(24):
            p = df[(df['scanner']==i)][['beacon',f'a{j}r{r}']]
            for q in range(len(p)):
                x = np.array([i for i in p[f'a{j}r{r}']])-np.array([i for i in p['beacon'].loc[p.index[q]]])
                y = pd.DataFrame([i for i in df[(df['scanner']==0)]['beacon'].to_numpy()])
                z = np.array([ i for i in pd.concat([y, pd.DataFrame(x)]).to_numpy()[:,0]])
                if z.size - np.unique(z,axis=0).size >= 12:
                    merger.append((i,j,r,q))
                    winning = True
                    break
            if winning:
                break
        if winning:
            break

2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36


In [391]:
merger

[(2, 19, 20, 4), (2, 19, 21, 4), (2, 19, 22, 4), (2, 19, 23, 4)]

In [390]:
df['scanner'].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36])

In [161]:
 s - s[b2] + s1[b1]

0     [-535, -526, -765]
1       [627, -311, 420]
2      [-587, -471, 242]
3      [581, -596, -710]
4       [-602, 794, 452]
5      [-716, 825, -607]
6       [487, 760, -636]
7      [-620, -456, 271]
8         [-37, -16, -1]
9     [-647, -543, -606]
10     [608, -713, -821]
11      [-657, 903, 537]
12    [-667, -557, -727]
13      [670, -310, 354]
14     [-672, -675, 243]
15       [536, 888, 514]
16     [549, -605, -861]
17       [644, 831, 538]
18      [599, -378, 391]
19     [-641, 962, -643]
20       [133, 68, -113]
21      [516, 800, -553]
22       [582, 762, 538]
23     [-710, 842, -633]
24      [568, 907, -594]
25      [-549, 867, 508]
Name: beacon, dtype: object